In [1]:
import numpy as np
import pandas as pd
import pandas_datareader as pdr
from datetime import datetime
import bqplot.pyplot as plt
import bqplot.marks as mk
import strategies as st
import backtest as bt

import technical_indicators as ti

%matplotlib inline

In [2]:
# ada_usd = pdr.DataReader('ada-usd', start=datetime(2021, 4, 12), end=datetime(2021, 4, 12), data_source='yahoo')

In [3]:
ada_usd = pd.read_csv('data/yf_ada_usd_5y_20210413.csv')
ada_usd['Date'] = pd.to_datetime(ada_usd['Date'])
ada_usd.set_index('Date', inplace=True)

In [4]:
xrp_usd = pd.read_csv('data/yf_xrp_usd_5y_20210415.csv')
xrp_usd['Date'] = pd.to_datetime(xrp_usd['Date'])
xrp_usd.set_index('Date', inplace=True)

In [5]:
ada_usd.dropna(inplace=True)

In [6]:
xrp_usd.dropna(inplace=True)

In [7]:
ada_usd_2021 = ada_usd.loc['2021-01-01' : '2021-04-13']

In [8]:
fig = plt.figure(title='ada-usd')

fig.layout.width = '800px'

ohlc = plt.ohlc(x=ada_usd_2021.index, y=ada_usd_2021[['Open', 'High', 'Low', 'Close']],
                marker='candle', stroke='blue')

ohlc.colors=['lime', 'tomato']
plt.xlabel('Date')

plt.show()

In [9]:
# ada_usd['TR'] = ti.calc_true_range(ada_usd)

In [10]:
# ada_usd['ATR'] = ti.calc_average_true_range(ada_usd, 'Low', 'High', 'Adj Close', 14)

In [11]:
# ada_usd['streak'] = ti.find_streak(ada_usd, 'Adj Close')[0]

In [12]:
# ada_usd['min_4'] = ti.calc_min_n(ada_usd, 'Close', 4)

In [13]:
# ada_usd['id'] = ti.find_inday(ada_usd, 'Low', 'High')

In [14]:
# ada_usd['nr4'] = ti.find_nr4(ada_usd, 'Low', 'High', 'Adj Close')

In [15]:
id_nr4 = st.find_id_nr4(ada_usd)

In [16]:
id_nr4.head()

,Price,Stop,Executed
Date,,,
2017-10-05,0.022806,0.007082,False
2017-10-10,0.022807,0.003201,False
2017-10-11,0.022446,0.002387,False
2017-10-20,0.027112,0.002762,False
2017-10-27,0.027560,0.002248,False


In [17]:
ada_usd.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2017-10-01,0.021678,0.032226,0.017354,0.024969,0.024969,50068700.0
2017-10-02,0.024607,0.030088,0.019969,0.025932,0.025932,57641300.0
2017-10-03,0.025757,0.027425,0.020690,0.020816,0.020816,16997800.0
2017-10-04,0.020864,0.022806,0.020864,0.021931,0.021931,9000050.0
2017-10-05,0.021951,0.022154,0.020859,0.021489,0.021489,5562510.0


In [18]:
ada_usd[ada_usd.index=='2017-10-01'].index

DatetimeIndex(['2017-10-01'], dtype='datetime64[ns]', name='Date', freq=None)

In [19]:
float(id_nr4[id_nr4.index=='2017-10-05']['Price'])

0.022806

In [20]:
transactions = bt.open_orders(ada_usd, id_nr4, 'id_nr4', 'ada_usd')

In [21]:
transactions.head()

,Strategy,Product,Transaction,Price,Stop,Close Date
Date,,,,,,
2017-10-11,id_nr4,ada_usd,Buy,0.022446,0.002387,NaN
2017-10-20,id_nr4,ada_usd,Buy,0.027112,0.002762,NaN
2017-11-14,id_nr4,ada_usd,Buy,0.026300,0.004367,NaN
2017-12-27,id_nr4,ada_usd,Buy,0.430086,0.059717,NaN
2018-01-02,id_nr4,ada_usd,Buy,0.730051,0.114503,NaN


In [22]:
id_nr4.head()

,Price,Stop,Executed
Date,,,
2017-10-05,0.022806,0.007082,False
2017-10-10,0.022807,0.003201,False
2017-10-11,0.022446,0.002387,True
2017-10-20,0.027112,0.002762,True
2017-10-27,0.027560,0.002248,False


In [23]:
row = list(ada_usd.iterrows())[20]
current_date = row[0]
previous_date = (current_date + pd.DateOffset(days=-1)).strftime('%Y-%m-%d')

ix = (transactions.index < current_date)  & (transactions['Close Date'].isna())

# past_transactions = transactions[]

print(f'current_date: {current_date}')
#past_transactions['Days Open'] = (current_date - past_transactions.index).days

# print(f'past_transactions: {past_transactions}')

# open_date = past_transactions.index
# days_open = past_transactions['Days Open']

# print(f'open_date: {open_date}')
# print(f'days_open: {days_open}')

#range = pd.date_range(open_date[0], periods=days_open)

#ada_usd.loc[str(range[0].date()):str(range[-1].date()), 'High'].max()

# past_transactions['max_price'] = max(list(ada_usd.loc['2017-10-11':previous_date, 'High']))

# past_transactions

def retrieve_max_price(from_date, to_date):
    mp = max(list(ada_usd.loc[from_date:to_date, 'High']))
    return mp

# transactions.loc[ix,'max_price'] = transactions[ix].apply(lambda r: retrieve_max_price(r.name, previous_date), axis=1)

# past_transactions.apply(lambda r: r.name, axis=1)

# transactions

current_date: 2017-10-21 00:00:00


In [24]:
bt.estimate_profits(ada_usd, transactions)


current_date: 2017-10-01 00:00:00
past_transactions:
 Empty DataFrame
Columns: [Strategy, Product, Transaction, Price, Stop, Close Date, Max Price, Sales Price, Sales]
Index: []

current_date: 2017-10-02 00:00:00
past_transactions:
 Empty DataFrame
Columns: [Strategy, Product, Transaction, Price, Stop, Close Date, Max Price, Sales Price, Sales]
Index: []

current_date: 2017-10-03 00:00:00
past_transactions:
 Empty DataFrame
Columns: [Strategy, Product, Transaction, Price, Stop, Close Date, Max Price, Sales Price, Sales]
Index: []

current_date: 2017-10-04 00:00:00
past_transactions:
 Empty DataFrame
Columns: [Strategy, Product, Transaction, Price, Stop, Close Date, Max Price, Sales Price, Sales]
Index: []

current_date: 2017-10-05 00:00:00
past_transactions:
 Empty DataFrame
Columns: [Strategy, Product, Transaction, Price, Stop, Close Date, Max Price, Sales Price, Sales]
Index: []

current_date: 2017-10-06 00:00:00
past_transactions:
 Empty DataFrame
Columns: [Strategy, Product, Trans

current_date: 2017-11-11 00:00:00
past_transactions:
 Empty DataFrame
Columns: [Strategy, Product, Transaction, Price, Stop, Close Date, Max Price, Sales Price, Sales]
Index: []

current_date: 2017-11-12 00:00:00
past_transactions:
 Empty DataFrame
Columns: [Strategy, Product, Transaction, Price, Stop, Close Date, Max Price, Sales Price, Sales]
Index: []

current_date: 2017-11-13 00:00:00
past_transactions:
 Empty DataFrame
Columns: [Strategy, Product, Transaction, Price, Stop, Close Date, Max Price, Sales Price, Sales]
Index: []

current_date: 2017-11-14 00:00:00
past_transactions:
 Empty DataFrame
Columns: [Strategy, Product, Transaction, Price, Stop, Close Date, Max Price, Sales Price, Sales]
Index: []

current_date: 2017-11-15 00:00:00
past_transactions:
            Strategy  Product Transaction   Price      Stop Close Date  \
Date                                                                    
2017-11-14   id_nr4  ada_usd         Buy  0.0263  0.004367        NaN   

          

current_date: 2018-01-06 00:00:00
past_transactions:
 Empty DataFrame
Columns: [Strategy, Product, Transaction, Price, Stop, Close Date, Max Price, Sales Price, Sales]
Index: []

current_date: 2018-01-07 00:00:00
past_transactions:
 Empty DataFrame
Columns: [Strategy, Product, Transaction, Price, Stop, Close Date, Max Price, Sales Price, Sales]
Index: []

current_date: 2018-01-08 00:00:00
past_transactions:
 Empty DataFrame
Columns: [Strategy, Product, Transaction, Price, Stop, Close Date, Max Price, Sales Price, Sales]
Index: []

current_date: 2018-01-09 00:00:00
past_transactions:
 Empty DataFrame
Columns: [Strategy, Product, Transaction, Price, Stop, Close Date, Max Price, Sales Price, Sales]
Index: []

current_date: 2018-01-10 00:00:00
past_transactions:
 Empty DataFrame
Columns: [Strategy, Product, Transaction, Price, Stop, Close Date, Max Price, Sales Price, Sales]
Index: []

current_date: 2018-01-11 00:00:00
past_transactions:
 Empty DataFrame
Columns: [Strategy, Product, Transa

2018-02-17  2018-02-18 00:00:00   0.428074     0.401366  True  

current_date: 2018-02-19 00:00:00
past_transactions:
 Empty DataFrame
Columns: [Strategy, Product, Transaction, Price, Stop, Close Date, Max Price, Sales Price, Sales]
Index: []

current_date: 2018-02-20 00:00:00
past_transactions:
 Empty DataFrame
Columns: [Strategy, Product, Transaction, Price, Stop, Close Date, Max Price, Sales Price, Sales]
Index: []

current_date: 2018-02-21 00:00:00
past_transactions:
            Strategy  Product Transaction     Price      Stop  \
Date                                                           
2018-02-20   id_nr4  ada_usd         Buy  0.396608  0.023469   

                     Close Date  Max Price  Sales Price Sales  
Date                                                           
2018-02-20  2018-02-21 00:00:00   0.404329      0.38086  True  

current_date: 2018-02-22 00:00:00
past_transactions:
 Empty DataFrame
Columns: [Strategy, Product, Transaction, Price, Stop, Close Date, 

current_date: 2018-04-15 00:00:00
past_transactions:
 Empty DataFrame
Columns: [Strategy, Product, Transaction, Price, Stop, Close Date, Max Price, Sales Price, Sales]
Index: []

current_date: 2018-04-16 00:00:00
past_transactions:
 Empty DataFrame
Columns: [Strategy, Product, Transaction, Price, Stop, Close Date, Max Price, Sales Price, Sales]
Index: []

current_date: 2018-04-17 00:00:00
past_transactions:
 Empty DataFrame
Columns: [Strategy, Product, Transaction, Price, Stop, Close Date, Max Price, Sales Price, Sales]
Index: []

current_date: 2018-04-18 00:00:00
past_transactions:
 Empty DataFrame
Columns: [Strategy, Product, Transaction, Price, Stop, Close Date, Max Price, Sales Price, Sales]
Index: []

current_date: 2018-04-19 00:00:00
past_transactions:
 Empty DataFrame
Columns: [Strategy, Product, Transaction, Price, Stop, Close Date, Max Price, Sales Price, Sales]
Index: []

current_date: 2018-04-20 00:00:00
past_transactions:
 Empty DataFrame
Columns: [Strategy, Product, Transa

...

current_date: 2020-10-06 00:00:00
past_transactions:
            Strategy  Product Transaction     Price      Stop  \
Date                                                           
2020-10-04   id_nr4  ada_usd         Buy  0.095158  0.005648   

                     Close Date  Max Price  Sales Price Sales  
Date                                                           
2020-10-04  2020-10-06 00:00:00   0.098844     0.093196  True  

current_date: 2020-10-07 00:00:00
past_transactions:
 Empty DataFrame
Columns: [Strategy, Product, Transaction, Price, Stop, Close Date, Max Price, Sales Price, Sales]
Index: []

current_date: 2020-10-08 00:00:00
past_transactions:
 Empty DataFrame
Columns: [Strategy, Product, Transaction, Price, Stop, Close Date, Max Price, Sales Price, Sales]
Index: []

current_date: 2020-10-10 00:00:00
past_transactions:
 Empty DataFrame
Columns: [Strategy, Product, Transaction, Price, Stop, Close Date, Max Price, Sales Price, Sales]
Index: []

current_date: 2020-10-1

2020-11-20  2020-11-22 00:00:00   0.133603     0.128128  True  

current_date: 2020-11-23 00:00:00
past_transactions:
 Empty DataFrame
Columns: [Strategy, Product, Transaction, Price, Stop, Close Date, Max Price, Sales Price, Sales]
Index: []

current_date: 2020-11-24 00:00:00
past_transactions:
 Empty DataFrame
Columns: [Strategy, Product, Transaction, Price, Stop, Close Date, Max Price, Sales Price, Sales]
Index: []

current_date: 2020-11-25 00:00:00
past_transactions:
 Empty DataFrame
Columns: [Strategy, Product, Transaction, Price, Stop, Close Date, Max Price, Sales Price, Sales]
Index: []

current_date: 2020-11-26 00:00:00
past_transactions:
 Empty DataFrame
Columns: [Strategy, Product, Transaction, Price, Stop, Close Date, Max Price, Sales Price, Sales]
Index: []

current_date: 2020-11-27 00:00:00
past_transactions:
 Empty DataFrame
Columns: [Strategy, Product, Transaction, Price, Stop, Close Date, Max Price, Sales Price, Sales]
Index: []

current_date: 2020-11-28 00:00:00
past_t

current_date: 2021-01-16 00:00:00
past_transactions:
 Empty DataFrame
Columns: [Strategy, Product, Transaction, Price, Stop, Close Date, Max Price, Sales Price, Sales]
Index: []

current_date: 2021-01-17 00:00:00
past_transactions:
 Empty DataFrame
Columns: [Strategy, Product, Transaction, Price, Stop, Close Date, Max Price, Sales Price, Sales]
Index: []

current_date: 2021-01-18 00:00:00
past_transactions:
 Empty DataFrame
Columns: [Strategy, Product, Transaction, Price, Stop, Close Date, Max Price, Sales Price, Sales]
Index: []

current_date: 2021-01-19 00:00:00
past_transactions:
 Empty DataFrame
Columns: [Strategy, Product, Transaction, Price, Stop, Close Date, Max Price, Sales Price, Sales]
Index: []

current_date: 2021-01-20 00:00:00
past_transactions:
 Empty DataFrame
Columns: [Strategy, Product, Transaction, Price, Stop, Close Date, Max Price, Sales Price, Sales]
Index: []

current_date: 2021-01-21 00:00:00
past_transactions:
 Empty DataFrame
Columns: [Strategy, Product, Transa

current_date: 2021-03-04 00:00:00
past_transactions:
 Empty DataFrame
Columns: [Strategy, Product, Transaction, Price, Stop, Close Date, Max Price, Sales Price, Sales]
Index: []

current_date: 2021-03-05 00:00:00
past_transactions:
 Empty DataFrame
Columns: [Strategy, Product, Transaction, Price, Stop, Close Date, Max Price, Sales Price, Sales]
Index: []

current_date: 2021-03-06 00:00:00
past_transactions:
 Empty DataFrame
Columns: [Strategy, Product, Transaction, Price, Stop, Close Date, Max Price, Sales Price, Sales]
Index: []

current_date: 2021-03-07 00:00:00
past_transactions:
 Empty DataFrame
Columns: [Strategy, Product, Transaction, Price, Stop, Close Date, Max Price, Sales Price, Sales]
Index: []

current_date: 2021-03-08 00:00:00
past_transactions:
 Empty DataFrame
Columns: [Strategy, Product, Transaction, Price, Stop, Close Date, Max Price, Sales Price, Sales]
Index: []

current_date: 2021-03-09 00:00:00
past_transactions:
            Strategy  Product Transaction     Price 

2021-03-29   1.245915     1.160077  False  

current_date: 2021-04-06 00:00:00
past_transactions:
            Strategy  Product Transaction     Price      Stop Close Date  \
Date                                                                      
2021-03-29   id_nr4  ada_usd         Buy  1.219583  0.085838        NaN   

            Max Price  Sales Price  Sales  
Date                                       
2021-03-29   1.245915     1.160077  False  

current_date: 2021-04-07 00:00:00
past_transactions:
            Strategy  Product Transaction     Price      Stop  \
Date                                                           
2021-03-29   id_nr4  ada_usd         Buy  1.219583  0.085838   

                     Close Date  Max Price  Sales Price Sales  
Date                                                           
2021-03-29  2021-04-07 00:00:00   1.340895     1.255057  True  

current_date: 2021-04-08 00:00:00
past_transactions:
 Empty DataFrame
Columns: [Strategy, Product, Tra

In [25]:
entry_date = '2020-07-25'

entry_price = ada_usd[entry_date:entry_date]['Buy Price']
last_atr = ada_usd[entry_date:entry_date]['Last ATR']

plot_data = ada_usd[entry_date:'2020-09-25']

stop = entry_price - 3 * last_atr
top = entry_price + 3 * last_atr

fig = plt.figure(title='ada-usd')

fig.layout.width = '800px'

plt.ohlc(x=plot_data.index, y=plot_data[['Open', 'High', 'Low', 'Close']],
                marker='candle', stroke='blue')


plt.plot(x=plot_data.index, y=np.full(plot_data.shape[0], entry_price))
plt.plot(x=plot_data.index, y=np.full(plot_data.shape[0], stop))
plt.plot(x=plot_data.index, y=np.full(plot_data.shape[0], top))

ohlc.colors=['lime', 'tomato']
plt.xlabel('Date')

plt.show()

KeyError: 'Buy Price'

In [ ]:
df = pd.DataFrame(columns=['Date', 'Product', 'Transaction', 'Price', 'Stop'])

In [ ]:
df = df.append({'Date': '2021-04-23', 'Product': 'ada-usd', 'Transaction': 'Buy', 'Price': 15, 'Stop': 10}, ignore_index=True)

In [ ]:
df

In [27]:
transactions

,Strategy,Product,Transaction,Price,Stop,Close Date,Max Price,Sales Price,Sales
Date,,,,,,,,,
2017-10-11,id_nr4,ada_usd,Buy,0.022446,0.002387,2017-10-14 00:00:00,0.033816,0.031429,True
2017-10-20,id_nr4,ada_usd,Buy,0.027112,0.002762,2017-10-21 00:00:00,0.030855,0.028093,True
2017-11-14,id_nr4,ada_usd,Buy,0.026300,0.004367,2017-11-21 00:00:00,0.032300,0.027933,True
2017-12-27,id_nr4,ada_usd,Buy,0.430086,0.059717,2017-12-28 00:00:00,0.430827,0.371110,True
2018-01-02,id_nr4,ada_usd,Buy,0.730051,0.114503,2018-01-05 00:00:00,1.327210,1.212707,True
...,...,...,...,...,...,...,...,...,...
2021-01-24,id_nr4,ada_usd,Buy,0.357211,0.045634,2021-01-27 00:00:00,0.368639,0.323005,True
2021-02-10,id_nr4,ada_usd,Buy,0.709040,0.074012,2021-02-12 00:00:00,0.973325,0.899313,True
2021-03-08,id_nr4,ada_usd,Buy,1.142286,0.085434,2021-03-10 00:00:00,1.214923,1.129489,True
